In [16]:
# ============================================
# CS-340 Project Two - Grazioso Salvare
# ============================================

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Standard Python / data tools
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# CRUD module for MongoDB
from crud import CRUD


###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "033197"

# Connect to database via CRUD module
db = CRUD(username, password)

# Read all documents from the animals collection
df = pd.DataFrame.from_records(db.read({}))

# Drop MongoDB _id field so Dash DataTable doesn't crash
df.drop(columns=['_id'], inplace=True, errors='ignore')

# Rescue filters based on CS-340 Dashboard Specifications
RESCUE_FILTERS = {
    "Water Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": [
            "Labrador Retriever Mix",
            "Chesapeake Bay Retriever",
            "Newfoundland"
        ]},
        "sex_upon_outcome": "Intact Female",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },

    "Mountain or Wilderness Rescue": {
        "animal_type": "Dog",
        "breed": {"$in": [
            "German Shepherd",
            "Alaskan Malamute",
            "Old English Sheepdog",
            "Siberian Husky",
            "Rottweiler"
        ]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
    },

    "Disaster or Individual Tracking": {
        "animal_type": "Dog",
        "breed": {"$in": [
            "Doberman Pinscher",
            "German Shepherd",
            "Golden Retriever",
            "Bloodhound",
            "Rottweiler"
        ]},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
    }
}


#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

# Load and encode Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # ---------- Header with logo and your name ----------
    html.Div([
        html.Div([
            html.A(
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={'height': '80px'}
                ),
                href="https://www.snhu.edu",
                target="_blank"
            )
        ], style={'display': 'inline-block', 'verticalAlign': 'top'}),

        html.Div([
            html.H1('Grazioso Salvare Rescue Dog Dashboard'),
            html.P('Developed by: Jeremia Lim')
        ], style={'display': 'inline-block', 'marginLeft': '20px'})

    ], style={'textAlign': 'left'}),

    html.Hr(),

    # ---------- Rescue type filter ----------
    html.Div([
        html.Label('Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            inline=True
        )
    ], style={'marginBottom': '20px'}),

    html.Hr(),

    # ---------- Interactive data table ----------
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action='native',
        filter_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
        style_cell={
            'textAlign': 'left',
            'minWidth': '100px', 'width': '150px', 'maxWidth': '250px',
            'whiteSpace': 'normal'
        }
    ),

    html.Br(),
    html.Hr(),

    # ---------- Charts and map side by side ----------
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
                style={'width': '50%'}
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
                style={'width': '50%'}
            )
        ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################

# 1) Filter the table based on rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    """
    Updates the data table when the rescue type radio button changes.
    Uses MongoDB via the CRUD class (db.read).
    """
    if not filter_type or filter_type == 'Reset':
        records = db.read({})
    else:
        query = RESCUE_FILTERS.get(filter_type, {})
        records = db.read(query)

    dff = pd.DataFrame.from_records(records)
    dff.drop(columns=['_id'], inplace=True, errors='ignore')
    return dff.to_dict('records')


# 2) Chart based on what's currently in the table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    """
    Builds a pie chart of breed distribution from the visible table data.
    """
    if viewData is None:
        dff = df.copy()
    else:
        dff = pd.DataFrame(viewData)

    if dff.empty or 'breed' not in dff.columns:
        fig = px.bar(title="No data available")
    else:
        breed_counts = dff['breed'].value_counts().reset_index()
        breed_counts.columns = ['breed', 'count']
        fig = px.pie(
            breed_counts,
            names='breed',
            values='count',
            title='Breed Distribution for Selected Rescue Filter'
        )

    return [dcc.Graph(figure=fig)]


# 3) Highlight selected columns in the table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    """
    Highlights any selected column headers in the data table.
    """
    if not selected_columns:
        return []

    return [{
        'if': {'column_id': col},
        'background_color': '#D2F3FF'
    } for col in selected_columns]


# 4) Update the geo-location map for the selected row
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    """
    Updates the map marker based on the selected row in the data table.
    Uses location_lat / location_long columns.
    """
    if viewData is None:
        return []

    dff = pd.DataFrame.from_dict(viewData)

    if dff.empty:
        return []

    # If nothing selected, default to first row
    if not index:
        row = 0
    else:
        row = index[0]

    if row >= len(dff):
        row = 0

    # Fallback values in case columns are missing
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return []

    lat = dff.loc[row, 'location_lat']
    lon = dff.loc[row, 'location_long']
    breed = dff.loc[row, 'breed'] if 'breed' in dff.columns else 'Unknown'
    name = dff.loc[row, 'name'] if 'name' in dff.columns else 'Unknown'

    # Austin, TX approximate center
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(str(breed)),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(str(name))
                        ])
                    ]
                )
            ]
        )
    ]


# Run app and display result (external browser tab)
app.run_server()


Dash app running on https://cartooncrystal-fictionsamuel-3000.codio.io/proxy/8050/
